In [2]:
'''
#Train a simple deep CNN on the CIFAR10 small images dataset.

It gets to 75% validation accuracy in 25 epochs, and 79% after 50 epochs.
(it's still underfitting at that point, though).
'''

from __future__ import print_function
from keras.datasets import cifar10
from keras.utils import np_utils
import os

batch_size = 100
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 3, 32, 32)
50000 train samples
10000 test samples


In [3]:
import numpy as np
## changing to grayscale
def rgb2gray(rgb):
    r, g, b = rgb[:,0,:,:], rgb[:,1,:,:], rgb[:,2,:,:]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b
    gray = np.expand_dims(gray, axis=3)
    print(gray.shape)
    return gray

In [4]:
x_train=rgb2gray(x_train)
x_test=rgb2gray(x_test)

(50000, 32, 32, 1)
(10000, 32, 32, 1)


In [5]:
## normalize/scale
x_train /= 255
x_test /= 255
X_train, y_train = (x_train, y_train)

In [7]:
import tensorflow as tf
EPOCHS = 100
BATCH_SIZE = 100

In [8]:

def LeNet(x):    
    # Hyperparameters
    mu = 0
    sigma = 0.1
    layer_depth = {
        'layer_1' : 6,
        'layer_2' : 16,
        'layer_3' : 120,
        'layer_f1' : 84
    }

    
    # TODO: Layer 1: Convolutional. Input = 32x32x1. Output = 28x28x6.
    conv1_w = tf.Variable(tf.truncated_normal(shape = [5,5,1,6],mean = mu, stddev = sigma))
    conv1_b = tf.Variable(tf.zeros([6]))
    conv1 = tf.nn.conv2d(x,conv1_w, strides = [1,1,1,1], padding = 'VALID') + conv1_b 
    # TODO: Activation.
    conv1 = tf.nn.relu(conv1)

    # TODO: Pooling. Input = 28x28x6. Output = 14x14x6.
    pool_1 = tf.nn.max_pool(conv1,ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'VALID')
    
    # TODO: Layer 2: Convolutional. Output = 10x10x16.
    conv2_w = tf.Variable(tf.truncated_normal(shape = [5,5,6,16], mean = mu, stddev = sigma))
    conv2_b = tf.Variable(tf.zeros([16]))
    conv2 = tf.nn.conv2d(pool_1, conv2_w, strides = [1,1,1,1], padding = 'VALID') + conv2_b
    # TODO: Activation.
    conv2 = tf.nn.relu(conv2)

    # TODO: Pooling. Input = 10x10x16. Output = 5x5x16.
    pool_2 = tf.nn.max_pool(conv2, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'VALID') 
    
    # TODO: Flatten. Input = 5x5x16. Output = 400.
    fc1 = tf.reshape(pool_2, [-1, 400])
    
    # TODO: Layer 3: Fully Connected. Input = 400. Output = 120.
    fc1_w = tf.Variable(tf.truncated_normal(shape = (400,120), mean = mu, stddev = sigma))
    fc1_b = tf.Variable(tf.zeros([120]))
    fc1 = tf.matmul(fc1,fc1_w) + fc1_b
    
    # TODO: Activation.
    fc1 = tf.nn.relu(fc1)

    # TODO: Layer 4: Fully Connected. Input = 120. Output = 84.
    fc2_w = tf.Variable(tf.truncated_normal(shape = (120,84), mean = mu, stddev = sigma))
    fc2_b = tf.Variable(tf.zeros([84]))
    fc2 = tf.matmul(fc1,fc2_w) + fc2_b
    # TODO: Activation.
    fc2 = tf.nn.relu(fc2)
    
    # TODO: Layer 5: Fully Connected. Input = 84. Output = 10.
    fc3_w = tf.Variable(tf.truncated_normal(shape = (84,10), mean = mu , stddev = sigma))
    fc3_b = tf.Variable(tf.zeros([10]))
    logits = tf.matmul(fc2, fc3_w) + fc3_b
    # Create the model
    # y = tf.nn.softmax(tf.matmul(fc2, fc3_w) + fc3_b)
    return logits

In [9]:
x = tf.placeholder(tf.float32, (None, 32, 32, 1))
y = tf.placeholder(tf.float32, (None, 10))
one_hot_y = y

In [10]:
rate = 0.1

## initialize otimizer
logits = LeNet(x)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = y)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.MomentumOptimizer(learning_rate = rate, momentum = 0.5)
training_operation = optimizer.minimize(loss_operation)

In [11]:
## evaluation function
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

In [12]:
print(X_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(50000, 32, 32, 1)
(50000, 10)
(10000, 32, 32, 1)
(10000, 10)


In [13]:
init = tf.initialize_all_variables()
with tf.Session() as sess:
    sess.run(init)
    num_examples = len(X_train)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        for offset in range(0, num_examples, 1000):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y})
            
        validation_accuracy = evaluate(x_test, y_test)
        print("EPOCH {} ...".format(i+1))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()
        
    saver.save(sess, './lenet')
    print("Model saved")

Training...

EPOCH 1 ...
Validation Accuracy = 0.154

EPOCH 2 ...
Validation Accuracy = 0.190

EPOCH 3 ...
Validation Accuracy = 0.248

EPOCH 4 ...
Validation Accuracy = 0.262

EPOCH 5 ...
Validation Accuracy = 0.261

EPOCH 6 ...
Validation Accuracy = 0.308

EPOCH 7 ...
Validation Accuracy = 0.260

EPOCH 8 ...
Validation Accuracy = 0.314

EPOCH 9 ...
Validation Accuracy = 0.318

EPOCH 10 ...
Validation Accuracy = 0.319

EPOCH 11 ...
Validation Accuracy = 0.334

EPOCH 12 ...
Validation Accuracy = 0.324

EPOCH 13 ...
Validation Accuracy = 0.308

EPOCH 14 ...
Validation Accuracy = 0.334

EPOCH 15 ...
Validation Accuracy = 0.340

EPOCH 16 ...
Validation Accuracy = 0.346

EPOCH 17 ...
Validation Accuracy = 0.335

EPOCH 18 ...
Validation Accuracy = 0.337

EPOCH 19 ...
Validation Accuracy = 0.341

EPOCH 20 ...
Validation Accuracy = 0.347

EPOCH 21 ...
Validation Accuracy = 0.339

EPOCH 22 ...
Validation Accuracy = 0.345

EPOCH 23 ...
Validation Accuracy = 0.340



KeyboardInterrupt: 